In [ ]:
!pip install --upgrade progressbar2

  Attempting uninstall: progressbar2
    Found existing installation: progressbar2 3.38.0
    Uninstalling progressbar2-3.38.0:
      Successfully uninstalled progressbar2-3.38.0


In [ ]:
# Define root directory from Google Drive
from google.colab import drive
import os
drive.mount('/content/drive')
drive.mount("/content/drive", force_remount=True)
os.chdir("/content/drive/My Drive/trustpilot")
os.listdir()

Mounted at /content/drive
Mounted at /content/drive


['united_states.auto-adjusted_gender.geocoded.json',
 'france.auto-adjusted_gender.NUTS-regions.json',
 'france.json',
 'trustpilot.pt',
 '.ipynb_checkpoints',
 'united_states.json',
 'trustpilot_france.pt',
 'trustpilot_usa.pt',
 'trustpilot_usa_bert.pt']

In [ ]:
# Import Torch
import torch
# Setup device
dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(dev)

cuda


In [ ]:
import json
  
# Opening JSON file
f = open('united_states.json',)
  
# returns JSON object as 
# a dictionary
data = json.load(f)
  
# Closing file
f.close()

In [ ]:
usa_reviews = []
usa_targets = []

In [ ]:
from string import punctuation
print(len(data))
count = 0
for i in range(0, len(data)):
  if (data[i]['reviews'][0]['text'] != [] and (''.join([c for c in data[i]['reviews'][0]['text'][0] if c not in punctuation]))!= '' and count < 30000):
    usa_reviews.append(data[i]['reviews'][0]['text'][0])
    usa_targets.append(int(data[i]['reviews'][0]['rating']))
    count = count + 1

648784


In [ ]:
# Remove punctuaction
for i in range(0, len(usa_reviews)):
  usa_reviews[i] = ''.join([c for c in usa_reviews[i] if c not in punctuation])

In [ ]:
# Split words
usa_reviews = [[w for w in r.split(" ") if len(w) > 0] for r in usa_reviews if len(r) > 0]

In [ ]:
print(len(usa_reviews))
print(len(usa_targets))
print(usa_reviews[10])
print(usa_targets[10])

30000
30000
['Have', 'been', 'impressed', 'with', 'my', 'most', 'recent', 'order', 'as', 'usually', 'i', 'have', 'to', 'wait', 'a', 'couple', 'weeks', 'before', 'i', 'get', 'mine', 'as', 'its', 'quite', 'a', 'specific', 'prescription', 'Have', 'alway', 'supplied', 'a', 'top', 'quality', 'products', 'and', 'service', 'since', 'i', 'started', 'using', 'them', '2', 'years', 'ago', 'Would', 'recommend', 'to', 'all']
5


In [ ]:
# Print average review length
review_lens_usa = [len(r) for r in usa_reviews]
print(sum(review_lens_usa)/len(review_lens_usa))

34.4464


In [ ]:
# Build vocabulary
# Start index = 1, we need 0 to pad sequences
words_usa = list(set([w for r in usa_reviews for w in r]))
vocab_usa = {words_usa[i]: i+1 for i in range(len(words_usa))}

In [ ]:
len(vocab_usa)

40894

In [ ]:
# Convert reviews to word indexes
usa_reviews = [[vocab_usa[w] for w in r] for r in usa_reviews]

In [ ]:
# Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import urllib

# Parameters
seq_len = 30
frac_train = 0.8
frac_val = 0.1
batch_size = 64

In [ ]:
# Clip reviews to max seq_len words
usa_reviews = [r[:seq_len] for r in usa_reviews]

In [ ]:
# Print average review length now
review_lens = [len(r) for r in usa_reviews]
print(sum(review_lens)/len(review_lens))

21.182033333333333


In [ ]:
# Pad reviews shorter than seq_len
usa_reviews = [[0]*(seq_len - len(r)) + r for r in usa_reviews]

In [ ]:
# Print average review length now
review_lens = [len(r) for r in usa_reviews]
print(sum(review_lens)/len(review_lens))

30.0


In [ ]:
# Convert reviews to tensor
data_usa = torch.LongTensor(usa_reviews)
print(data_usa.size())

torch.Size([30000, 30])


In [ ]:
# Load sentiments: less than 3 stars is negative (0), 4 and 5 are positive (1)   
sentiments_usa = [0 if f <= 3 else 1 for f in usa_targets]

print(len(sentiments_usa))

30000


In [ ]:
# Convert sentiments to tensor
labels_usa = torch.LongTensor(sentiments_usa)
labels_usa.size()

torch.Size([30000])

In [ ]:
# Shuffle dataset
num_data = data_usa.size(0)
shuffle_idx = torch.randperm(num_data)
data_usa = data_usa[shuffle_idx,:]
labels_usa = labels_usa[shuffle_idx]

In [ ]:
# Split training and test
num_train = int(num_data*frac_train)
num_val = int(num_data*frac_val)
num_test = num_data - num_train - num_val
train_data = data_usa[:num_train,:]
train_labels = labels_usa[:num_train]
val_data = data_usa[num_train:num_train+num_val,:]
val_labels = labels_usa[num_train:num_train+num_val]
test_data = data_usa[num_train+num_val:,:]
test_labels = labels_usa[num_train+num_val:]

In [ ]:
# Create datasets
train_dataset_usa = TensorDataset(train_data, train_labels)
val_dataset_usa = TensorDataset(val_data, val_labels)
test_dataset_usa = TensorDataset(test_data, test_labels)

In [ ]:
# Create loaders
train_loader_usa = DataLoader(train_dataset_usa, batch_size=batch_size, shuffle=True,  drop_last=True)
val_loader_usa = DataLoader(val_dataset_usa,   batch_size=batch_size, shuffle=False, drop_last=True)
test_loader_usa = DataLoader(test_dataset_usa,  batch_size=batch_size, shuffle=False, drop_last=True)

In [ ]:
import json
  
# Opening JSON file
f = open('france.json',)
  
# returns JSON object as 
# a dictionary
data = json.load(f)
  
# Closing file
f.close()

In [ ]:
france_reviews = []
france_targets = []

In [ ]:
from string import punctuation
print(len(data))
count = 0
for i in range(0, len(data)):
  if (data[i]['reviews'][0]['text'] != [] and (''.join([c for c in data[i]['reviews'][0]['text'][0] if c not in punctuation]))!= '' and count < 30000):
    france_reviews.append(data[i]['reviews'][0]['text'][0])
    france_targets.append(int(data[i]['reviews'][0]['rating']))
    count = count + 1

741237


In [ ]:
print(len(france_reviews))
print(len(france_targets))

30000
30000


In [ ]:
# Remove punctuaction
for i in range(0, len(france_reviews)):
  france_reviews[i] = ''.join([c for c in france_reviews[i] if c not in punctuation])

In [ ]:
count=0
for r in range(0, len(france_reviews)):
  if len(france_reviews[r])<= 0:
    print(r)
    count=count+1

print(count)

0


In [ ]:
# Split words
france_reviews = [[w for w in r.split(" ") if len(w) > 0] for r in france_reviews if len(r) > 0]

In [ ]:
# Print average review length
review_lens_france = [len(r) for r in france_reviews]
print(sum(review_lens_france)/len(review_lens_france))

19.232166666666668


In [ ]:
# Build vocabulary
# Start index = 1, we need 0 to pad sequences
words_france = list(set([w for r in france_reviews for w in r]))
vocab_france = {words_france[i]: i+1 for i in range(len(words_france))}

In [ ]:
# Convert reviews to word indexes
france_reviews = [[vocab_france[w] for w in r] for r in france_reviews]

In [ ]:
# Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import urllib

# Parameters
seq_len = 20
frac_train = 0.8
frac_val = 0.1
batch_size = 64

In [ ]:
# Clip reviews to max seq_len words
france_reviews = [r[:seq_len] for r in france_reviews]

In [ ]:
# Print average review length now
review_lens = [len(r) for r in france_reviews]
print(sum(review_lens)/len(review_lens))

12.9727


In [ ]:
# Pad reviews shorter than seq_len
france_reviews = [[0]*(seq_len - len(r)) + r for r in france_reviews]

In [ ]:
# Print average review length now
review_lens = [len(r) for r in france_reviews]
print(sum(review_lens)/len(review_lens))

20.0


In [ ]:
# Convert reviews to tensor
data_france = torch.LongTensor(france_reviews)
print(data_france.size())

torch.Size([30000, 20])


In [ ]:
# Load sentiments: less than 3 stars is negative (0), 4 and 5 are positive (1)   
sentiments_france = [0 if f <= 3 else 1 for f in france_targets]

print(len(sentiments_france))

30000


In [ ]:
print(sentiments_france[10])

1


In [ ]:
# Convert sentiments to tensor
labels_france = torch.LongTensor(sentiments_france)
labels_france.size()

torch.Size([30000])

In [ ]:
# Shuffle dataset
num_data = data_france.size(0)
shuffle_idx = torch.randperm(num_data)
data = data_france[shuffle_idx,:]
labels = labels_france[shuffle_idx]

In [ ]:
# Split training and test
num_train = int(num_data*frac_train)
num_val = int(num_data*frac_val)
num_test = num_data - num_train - num_val
train_data = data_france[:num_train,:]
train_labels = labels_france[:num_train]
val_data = data_france[num_train:num_train+num_val,:]
val_labels = labels_france[num_train:num_train+num_val]
test_data = data_france[num_train+num_val:,:]
test_labels = labels_france[num_train+num_val:]

In [ ]:
# Create datasets
train_dataset_france = TensorDataset(train_data, train_labels)
val_dataset_france = TensorDataset(val_data, val_labels)
test_dataset_france = TensorDataset(test_data, test_labels)

In [ ]:
# Create loaders
train_loader_france = DataLoader(train_dataset_france, batch_size=batch_size, shuffle=True,  drop_last=True)
val_loader_france = DataLoader(val_dataset_france,   batch_size=batch_size, shuffle=False, drop_last=True)
test_loader_france = DataLoader(test_dataset_france,  batch_size=batch_size, shuffle=False, drop_last=True)

In [ ]:
def merge_two_dicts(x, y):
    z = x.copy()   # start with keys and values of x
    z.update(y)    # modifies z with keys and values of y
    return z

In [ ]:
# Define model
class Model(nn.Module):
    
    def __init__(self, embed_size, rnn_size, vocab):
        # Call parent constructor
        super().__init__()
        # Store values
        self.rnn_size = rnn_size
        # Define modules
        self.embedding = nn.Embedding(len(vocab)+1, embed_size)
        self.rnn = nn.LSTMCell(embed_size, rnn_size)
        self.output = nn.Linear(rnn_size, 2)
        
    def forward(self, x):
        # Embed data
        x = self.embedding(x)
        # Initialize state
        c = x.new_zeros(x.shape[0], self.rnn_size)
        h = x.new_zeros(x.shape[0], self.rnn_size)
        # Input is: BxTxF
        # Process each time step
        for t in range(x.shape[1]):
            # Input at time t
            x_t = x[:,t,:]
            # Forward RNN and get new state
            h,c = self.rnn(x_t, (h, c))
        # Classify final state
        x = self.output(h)
        return x

In [ ]:
vocab = merge_two_dicts(vocab_usa, vocab_france)

In [ ]:
from collections import OrderedDict
SUM = lambda x,y : x+y
combo_fn = SUM

In [ ]:
# Define model
class ModelSUM(nn.Module):
    
    def __init__(self, embed_size, rnn_size, vocab, model1, model2):
        # Call parent constructor
        super().__init__()
        # Store values
        self.rnn_size = rnn_size
        # Define modules
        self.embedding = nn.Embedding(len(vocab)+1, embed_size)
        self.rnn = nn.LSTMCell(embed_size, rnn_size)
        self.output = nn.Linear(rnn_size, 2)

        summed_state_dict = OrderedDict()
        for key in self.state_dict():
          if key.find('rnn') >=0:
            summed_state_dict[key] = combo_fn(model1.state_dict()[key],model2.state_dict()[key])
          else:
            summed_state_dict[key] = self.state_dict()[key]
        self.load_state_dict(summed_state_dict)
        
    def forward(self, x):
        # Embed data
        x = self.embedding(x)
        # Initialize state
        c = x.new_zeros(x.shape[0], self.rnn_size)
        h = x.new_zeros(x.shape[0], self.rnn_size)
        # Input is: BxTxF
        # Process each time step
        for t in range(x.shape[1]):
            # Input at time t
            x_t = x[:,t,:]
            # Forward RNN and get new state
            h,c = self.rnn(x_t, (h, c))
        # Classify final state
        x = self.output(h)
        return x

In [ ]:
# Model parameters
embed_size = 1024
rnn_size = 512

In [ ]:
# Create model
model_usa = Model(embed_size, rnn_size, vocab_usa)
model_usa = model_usa.to(dev)

In [ ]:
#load model
model_name = 'trustpilot_usa.pt'
path = F"/content/drive/My Drive/trustpilot/{model_name}" 
model_usa.load_state_dict(torch.load(path))


<All keys matched successfully>

In [ ]:
# Create model
model_france = Model(embed_size, rnn_size, vocab_france)
model_france = model_france.to(dev)

In [ ]:
#load model
model_name = 'trustpilot_france.pt'
path = F"/content/drive/My Drive/trustpilot/{model_name}" 
model_france.load_state_dict(torch.load(path))


<All keys matched successfully>

In [ ]:
# Create model
model_sum = ModelSUM(embed_size, rnn_size, vocab, model_usa, model_france)
model_sum = model_sum.to(dev)

In [ ]:
import numpy as np

def eval_model(model, data_loader):
    model.eval() # Set model to eval mode
    true_preds, num_preds = 0., 0.

    with torch.no_grad(): # Deactivate gradients for the following code
        classes = {0, 1}
        count_zero = 0
        count_one = 0
        true_zero = 0
        true_one = 0
        for data_inputs, data_labels in data_loader:

            # Determine prediction of model on dev set
            preds = model(data_inputs.cuda())
            preds = preds.argmax(1)

            # Keep records of predictions for the accuracy metric (true_preds=TP+TN, num_preds=TP+TN+FP+FN)
            true_preds += (preds == data_labels.cuda()).sum()
            num_preds += data_labels.shape[0]
            
            for i in range(0, len(data_labels)):
              if data_labels[i] == 0:
                count_zero = count_zero+1
                if preds[i]==data_labels[i]:
                  true_zero = true_zero +1

              if data_labels[i] == 1:
                count_one = count_one+1
                if preds[i]==data_labels[i]:
                  true_one = true_one +1

    acc = true_preds / num_preds
    
    print(f'Class: {min(classes)}')
    print(f'Accuracy:{true_zero}/{count_zero}\n')
    print(f'Class: {max(classes)}')
    print(f'Accuracy:{true_one}/{count_one}\n')
    print("Accuracy of the model: %4.2f%%" % (100.0*acc))

In [ ]:
eval_model(model_sum, test_loader_usa)

Class: 0
Accuracy:53/263

Class: 1
Accuracy:2218/2681

Accuracy of the model: 77.14%


In [ ]:
eval_model(model_sum, test_loader_france)

Class: 0
Accuracy:100/299

Class: 1
Accuracy:1844/2645

Accuracy of the model: 66.03%
